In [11]:
import pandas as pd
from gensim import corpora, models, similarities
from collections import Counter

In [50]:
df = pd.read_pickle("../../pickles/dataframe_survey_2018-01-23_jung-liwc-enriched.pickle")
df = df[["text","func"]]

In [8]:
df.head(3)

,text,func
0,Jon Kågström playchilla.com uclassify.com abou...,n
1,❀*a drop of colour*❀ 1/39 next→ home ask past ...,n
2,Neko cool kids can't die home family daveblog ...,f


# Prepare Gensim model

In [10]:
# Create dictionary, no stopwords
texts = [[word for word in document.lower().split()]
          for ix, document in df.text.iteritems()]

In [14]:
texts[1][:4]

['❀*a', 'drop', 'of', 'colour*❀']

In [15]:
len(texts)

27959

In [16]:
# remove words that appear only once
freq = Counter()
for text in texts:
        for token in text:
             freq[token] += 1

In [19]:
texts = [[token for token in text if freq[token] > 1] for text in texts]

In [46]:
# Create Gensim dictionary
dictionary = corpora.Dictionary(texts)
dictionary.save('../../data/processed/typealyzer.dict') # store the dictionary, for future reference
print(dictionary.token2id("sleep"))

TypeError: 'dict' object is not callable

In [21]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [22]:
# TF-IDF model
tfidf = models.TfidfModel(corpus)

In [51]:
for func in df.func.unique():
    print("Function " + func)
    print("=========")
    
    textInputList = [] 
    for ix, series in df[df["func"] == func].iteritems():
        print(series.head(3))
        for ix, text in series.iteritems():
            for word in text.split():
                textInputList.append(word)

    print(textInputList[:2])
    subset_tfidf = tfidf[dictionary.doc2bow(textInputList)]
    # Top terms
    topt = sorted(subset_tfidf, key=lambda x: x[1], reverse=True)[:10]
    for i in topt:
        print(dictionary[i[0]])
    print('\n')

Function n
0    Jon Kågström playchilla.com uclassify.com abou...
1    ❀*a drop of colour*❀ 1/39 next→ home ask past ...
5    Perpetually In Transit Perpetually In Transit ...
Name: text, dtype: object
0    n
1    n
5    n
Name: func, dtype: object
['Jon', 'Kågström']
#
ago
n
,
notes
|
2014
/
to
the


Function f
2    Neko cool kids can't die home family daveblog ...
4    Codeode Free Applications Privacy Mantra (Free...
6    Moving on Moving on Chronicles of an Anglo/Spa...
Name: text, dtype: object
2    f
4    f
6    f
Name: func, dtype: object
['Neko', 'cool']
#
f
ago
,
notes
|
2014
/
·
-


Function t
3     Anything... Anything Mass Effect-related Music...
12    Gerald M. Weinberg: Writer - Consultant OFFICI...
13    Gerald M. Weinberg: Writer - Consultant OFFICI...
Name: text, dtype: object
3     t
12    t
13    t
Name: func, dtype: object
['Anything...', 'Anything']
#
ago
,
t
notes
|
/
2014
-
to


Function s
22    Oliver Brammer | I will blow your mind (but mo...
23    Divine Logis

TypeError: Can't convert 'float' object to str implicitly

In [57]:
for ix, text in df[df["func"] == "n"].iteritems():
    print(len(text.values))
    break

11359


In [33]:
s = df[df["func"] == "n"]
s.text.values

array([ "Jon Kågström playchilla.com uclassify.com about / cv Jon Kågström Game developer, text classification and some climbing. Posts RSS Comments RSS 17 Posts and 32 Comments till now Building a chatbot I always wanted to create a chatbot, but I never got around to actually doing it since I wouldn’t want to write anything where I have to explicitly give it knowledge or manually feed it with language rules. Every time I’ve started to sketch – I kind of got stuck being unable to understand how to overcome this. Lately I had another urge (after watching Prometheus) so once again I started by sketching on paper trying to understand how to attack the problem. Then, after a few evenings I decided just to start coding. Remembering the proverb “programming is understanding”. First I started by looking on Wikipedia, using classifiers to give contexts etc, but it became clear that it would be beneficial to start by looking on the syntactic structures of language. So I instead of jumping into 

In [40]:
df.func.unique()

array(['n', 'f', 't', 's', nan], dtype=object)